In [1]:
import mesa
from mesa import Model, Agent
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Users/oyama-kenshin/Desktop/Vault/05_Codes/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
class Participants(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.has_token = False
        self.bid_flag = False

        # 資産と効用
        self.wealth = self.model.base_wealth + int(self.random.normalvariate(0, 20))
        self.wealth = max(0, self.wealth)
    
    def calculate_bid(self, k, n):
        # W_k+1とL_kの計算
        w_k_plus_1 = self.w_1 - (self.w_1 - self.w_n) * (k / (n - 1))
        l_k = self.l_n_minus_1 * (k / (n - 1))

        #購入額の計算
        bid_price = w_k_plus_1 - l_k
        return bid_price
    
    def flag_if_interested(self, k):
        #k=self.model.sold_tokens
        n=self.model.num_participants

        if k >= self.model.num_data or self.has_token:
            return False

        #購入額の計算
        bid_price = self.calculate_bid(k, n)
        #print(f"Agent {self.unique_id} with wealth {self.wealth} bids {bid_price} when {k} tokens sold out of {n} participants.")

        if bid_price >= self.model.initial_price and self.model.initial_price <= self.wealth:
            self.bid_flag = True
            return True
        else:
            self.bid_flag = False
            return False

    def buy_if_interested(self):
            if self.bid_flag and not self.has_token and self.model.sold_tokens < self.model.num_data:
                # 購入
                self.wealth -= self.model.initial_price
                self.has_token = True
                self.model.sold_tokens += 1
                self.model.provider_revenue += self.model.initial_price
                self.bid_flag = False
                #print(f"Agent {self.unique_id} bought a token")
                return True
            else:
                #print(f"Agent {self.unique_id} did not buy a token")
                return False

In [56]:
class ResearcherAgent(Participants):
    """ペルソナ1：純粋な研究者"""
    def __init__(self, model):
        super().__init__(model)

        # 価値観パラメータ (個性を決定)
        # W_i(1) (独占時の効用) > W_i(N) (飽和時の効用) の制約を満たすように生成
        self.w_1 = self.random.normalvariate(150, 100)
        self.w_n = self.w_1 * self.random.uniform(0.80, 0.95) # 価値が落ちにくい

        # 効用が負にならないように調整
        self.w_1 = max(0, self.w_1)
        self.w_n = max(0, self.w_n)

        # L_i(N-1) (最大疎外感)
        self.l_0 = 0
        self.l_n_minus_1 = -self.random.normalvariate(120, 60) # 疎外感が小さい
        if self.l_n_minus_1 > 0:
            self.l_n_minus_1 = -self.l_n_minus_1

        #print(f"ResearcherAgent {self.unique_id} initialized with wealth {self.wealth}, w_1 {self.w_1}, w_n {self.w_n}, l_n-1 {self.l_n_minus_1}")

In [57]:
class CompetitorAgent(Participants):
    """ペルソナ2：競合企業のデータサイエンティスト"""
    def __init__(self, model):
        super().__init__(model)
        #企業なので、潤沢な資金を持つと想定
        self.wealth = int(self.model.base_wealth * self.random.uniform(1.5, 3.0))

        # 価値観パラメータ
        self.w_1 = self.random.normalvariate(100, 50)
        if self.w_1 < 0:
            self.w_1 = -self.w_1
        self.w_n = self.w_1 * self.random.uniform(0.1, 0.3) # 価値が落ちやすい

        # L_i(N-1) (最大疎外感)
        self.l_0 = 0
        self.l_n_minus_1 = -self.random.normalvariate(250, 50) # 疎外感が大きい
        if self.l_n_minus_1 > 0:
            self.l_n_minus_1 = -self.l_n_minus_1

    def calculate_bid(self, k, n):
        """
        L(k)がコンベックス（上に凸）になるようにBid計算をオーバーライド。
        kがある閾値を超えると、不利益（コスト）が爆発的に増加する。
        """
        # W(k+1)の計算は親クラスと同じ（線形）
        w_k_plus_1 = self.w_1 + (self.w_n - self.w_1) / (n - 1) * k if n > 1 else self.w_1

        # L(k)の計算を、kの2乗に比例するコンベックスな関数に変更
        # k/(n-1) は普及率のようなもの。普及率の2乗でコストが増加するとモデル化
        ratio = k / (n-1) if n > 1 else 0
        l_k = self.l_n_minus_1 * (ratio ** 2)

        return w_k_plus_1 - l_k


        #print(f"CompetitorAgent {self.unique_id} initialized with wealth {self.wealth}, w_1 {self.w_1}, w_n {self.w_n}, l_n-1 {self.l_n_minus_1}")

In [58]:
class DataMarket(Model):
    def __init__(self, agent_composition, num_data, initial_price, base_wealth, seed=None):
        super().__init__(seed=seed)
        self.num_data = num_data
        self.initial_price = initial_price
        self.base_wealth = base_wealth
        self.steps = 0

        #providerの設定
        self.provider_revenue = 0
        self.sold_tokens = 0

        self.datacollector = DataCollector(
            model_reporters={
                "Holders": lambda m: m.sold_tokens,
                "ProviderRevenue": "provider_revenue"
            }
        )
        self.datacollector.collect(self)

        # エージェントの生成
        self.participants = mesa.agent.AgentSet(agents=[], random=self.random)
        self.num_participants = 0
        # 渡された辞書に基づいて、各ペルソナを生成
        for agent_class, count in agent_composition.items():
            # 指定されたクラスのエージェントを、指定された数だけ生成
            agents = agent_class.create_agents(self, n=count)
            # 生成したエージェントをモデルのAgentSetに追加
            for a in agents:
                self.participants.add(a)
            # 総参加者数を更新
            self.num_participants += count    
    def step(self):
        k=self.sold_tokens
        non_holder_agents = self.participants.select(
            lambda agent: not agent.has_token
        )
        non_holder_agents.shuffle_do("flag_if_interested", k=k)
        non_holder_agents.shuffle_do("buy_if_interested")
        self.datacollector.collect(self)
        

In [59]:
test_model = DataMarket({ResearcherAgent: 10000, CompetitorAgent: 10000}, num_data=20000, initial_price = 200, base_wealth=550)

for _ in range(10):
    test_model.step()
    print(f"Step {_+1} complete. Total sold tokens: {test_model.sold_tokens}, Provider revenue: {test_model.provider_revenue}")

Step 1 complete. Total sold tokens: 3316, Provider revenue: 663200
Step 2 complete. Total sold tokens: 3915, Provider revenue: 783000
Step 3 complete. Total sold tokens: 4036, Provider revenue: 807200
Step 4 complete. Total sold tokens: 4054, Provider revenue: 810800
Step 5 complete. Total sold tokens: 4056, Provider revenue: 811200
Step 6 complete. Total sold tokens: 4057, Provider revenue: 811400
Step 7 complete. Total sold tokens: 4057, Provider revenue: 811400
Step 8 complete. Total sold tokens: 4057, Provider revenue: 811400
Step 9 complete. Total sold tokens: 4057, Provider revenue: 811400
Step 10 complete. Total sold tokens: 4057, Provider revenue: 811400
